In [7]:
import os
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import glob

# Definizione del percorso di input per le immagini e le etichette
input_dir_images = r'./data/imagesTrAug'
input_dir_labels = r'./data/labelsTrBW'

# Definizione del percorso di output per le immagini e le etichette
output_dir_images = r'./data/imagesTrFinal'
output_dir_labels = r'./data/labelsTrFinal'

# Creazione della directory di output se non esiste già
os.makedirs(output_dir_images, exist_ok=True)
os.makedirs(output_dir_labels, exist_ok=True)

# Numero totale desiderato di immagini generate
total_images_desired = 700

# Definizione dell'augmenter con diverse trasformazioni
augmenter = iaa.Sequential([
    iaa.Affine(
        scale=(0.8, 1.2),  # Zoom in/out
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # Traslazione
        rotate=(-45, 45),  # Rotazione
        shear=(-16, 16),  # Shear
    ),
    iaa.Fliplr(0.5),  # Ribalta orizzontalmente con una probabilità del 50%
    iaa.Flipud(0.5),  # Ribalta verticalmente con una probabilità del 50%
    # Aggiungi altre trasformazioni desiderate qui
])

# Scansiona tutte le immagini nella directory di input
image_files = sorted(glob.glob(os.path.join(input_dir_images, 'tnbc_*.png')))
label_files = sorted(glob.glob(os.path.join(input_dir_labels, 'tnbc_*.png')))

# Contatore per il numero totale di immagini generate
total_images_created = 101  # Inizia da 104

# Applica la data augmentation alle immagini e alle etichette
for image_path, label_path in zip(image_files, label_files):
    # Carica l'immagine
    image = np.array(Image.open(image_path))
    # Carica l'etichetta di segmentazione
    label = np.array(Image.open(label_path))

    # Applica la data augmentation sia all'immagine che all'etichetta
    augmented = augmenter(image=image, segmentation_maps=SegmentationMapsOnImage(label, shape=image.shape))
    augmented_image = augmented[0]
    augmented_label = augmented[1].get_arr_int()  # Converti in array numpy

    # Salva le immagini risultanti
    Image.fromarray(augmented_image).save(os.path.join(output_dir_images, f"tnbc_{total_images_created:03d}.png"))
    Image.fromarray(augmented_label).save(os.path.join(output_dir_labels, f"tnbc_{total_images_created:03d}.png"))

    total_images_created += 1

    # Esci se il numero totale di immagini generate è raggiunto
    if total_images_created >= 250:
        break

print("Data augmentation completata.")


Data augmentation completata.
